#  Système de Recommandation de Films

Ce notebook démontre un système de recommandation de films utilisant le filtrage collaboratif basé sur la similarité cosinus.  
Nous allons :
- Charger et prétraiter les données des films et des notes des utilisateurs.
- Calculer la similarité entre les films en fonction des genres.
- Calculer la similarité entre les utilisateurs en fonction de leurs évaluations.
- Implémenter un système de recommandation hybride combinant filtrage collaboratif et basé sur le contenu.
- Construire une interface simple avec Streamlit pour interagir avec le système.

In [94]:
import pandas as pd
import re
import requests


from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
from sklearn.preprocessing import MultiLabelBinarizer


##  Chargement des données
Nous chargeons deux fichiers CSV :
- `movies.csv` : contient les informations sur les films (titres, genres, etc.).
- `ratings.csv` : contient les notes attribuées par les utilisateurs aux films.

In [95]:
movies = pd.read_csv("/Users/ousmanediallo/PycharmProjects/Ai_projet/ai/movies.csv")
ratings = pd.read_csv("/Users/ousmanediallo/PycharmProjects/Ai_projet/ai/ratings.csv")

##  Prétraitement des genres des films
Nous transformons les genres en listes et les binarisons pour obtenir une matrice exploitable.

In [96]:
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movies['genres'])
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_, index=movies['movieId'])

##  Calcul de la similarité entre films
Nous utilisons la similarité cosinus pour comparer les films en fonction de leurs genres.

In [97]:
content_similarity = cosine_similarity(genre_df)
content_similarity_df = pd.DataFrame(content_similarity, index=movies['movieId'], columns=movies['movieId'])


##  Création d'une matrice utilisateur-film
Nous structurons les notes attribuées par les utilisateurs sous forme de matrice.


In [98]:
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
user_movie_matrix.fillna(0, inplace=True)


##  Calcul de la similarité entre utilisateurs
Nous utilisons la similarité cosinus pour comparer les utilisateurs en fonction de leurs notes.


In [99]:
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

##  Fonction pour nettoyer les titres de films
Permet de supprimer ', The' et l'année de sortie d'un titre de film.

In [100]:
def clean_movie_title(title):
    title = re.sub(r'\s*\(\d{4}\)$', '', title)  # Supprimer l'année
    match = re.match(r'^(.*), (The|A|An)$', title)
    return f"{match.group(2)} {match.group(1)}" if match else title

##  Fonction pour récupérer l'affiche d'un film
Nous utilisons l'API OMDb pour récupérer les affiches des films.

In [101]:
def get_movie_poster(title):
    api_key = "1d34d942"
    cleaned_title = clean_movie_title(title)
    url = f"http://www.omdbapi.com/?t={cleaned_title}&apikey={api_key}"
    response = requests.get(url).json()
    return response["Poster"] if "Poster" in response and response["Poster"] != "N/A" else "https://via.placeholder.com/150"

##  Système de recommandation hybride
Combine le filtrage collaboratif et basé sur le contenu pour recommander des films.

In [102]:
def HybridRecommender(user_id, top_n=3):
    similar_users = user_similarity_df[user_id].drop(user_id)
    similar_users = similar_users[similar_users > 0]
    if similar_users.empty:
        return []

    movie_scores = {}
    for similar_user, similarity in similar_users.items():
        rated_movies = user_movie_matrix.loc[similar_user][user_movie_matrix.loc[similar_user] > 0]
        for movie, rating in rated_movies.items():
            movie_scores[movie] = movie_scores.get(movie, 0) + rating * similarity

    movie_scores = pd.Series(movie_scores).sort_values(ascending=False)
    user_watched_movies = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id] > 0].index
    recommended_movie_ids = [movie for movie in movie_scores.index if movie not in user_watched_movies][:top_n]

    content_scores = {}
    for movie_id in recommended_movie_ids:
        similar_movies = content_similarity_df[movie_id].sort_values(ascending=False)
        similar_movies = similar_movies[similar_movies.index != movie_id].head(top_n)
        for similar_movie, similarity in similar_movies.items():
            if similar_movie not in recommended_movie_ids:
                content_scores[similar_movie] = content_scores.get(similar_movie, 0) + similarity

    hybrid_scores = pd.Series({**movie_scores.to_dict(), **content_scores}).sort_values(ascending=False)
    recommended_movies = movies[movies['movieId'].isin(hybrid_scores.index[:top_n])][['title', 'movieId']]

    return [(row['title'], get_movie_poster(row['title'])) for _, row in recommended_movies.iterrows()]

##  Interface utilisateur avec Streamlit
Interface simple permettant de générer des recommandations en entrant un ID utilisateur.

In [103]:
st.title("🎬 Système de Recommandation de Films")
user_id = st.number_input("Entrez votre ID utilisateur :", min_value=1, step=1)
if st.button("Recommander"):
    with st.spinner("Génération des recommandations..."):
        recommendations = HybridRecommender(user_id)
    if recommendations:
        st.subheader("📌 Films recommandés :")
        for title, poster_url in recommendations:
            st.image(poster_url, caption=title, width=200)
    else:
        st.warning("Aucune recommandation disponible pour cet utilisateur.")

2025-04-08 01:14:47.944 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 01:14:47.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 01:14:47.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 01:14:47.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 01:14:47.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 01:14:47.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 01:14:47.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 01:14:47.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

##  Affichage des affiches de films sans Streamlit
Utilisation de IPython pour afficher les affiches directement dans le notebook.

In [104]:
from IPython.display import Image, display

# Exemple d'utilisation dans un notebook
recommendations = HybridRecommender(user_id=1)
for title, poster_url in recommendations:
    print(f"Titre : {title}")
    display(Image(url=poster_url))

Titre : Pulp Fiction (1994)


Titre : Shawshank Redemption, The (1994)


Titre : Forrest Gump (1994)


## 🧪 Évaluation des performances du système de recommandation

Ce système a pour objectif de recommander des films pertinents à chaque utilisateur en combinant le filtrage collaboratif et le contenu des films (genres).

Pour mesurer la qualité des recommandations, nous utilisons trois métriques principales :

---

### 🎯 Précision

> Proportion de films recommandés qui ont été réellement appréciés par l'utilisateur.

$$
\text{Précision} = \frac{\text{Films correctement recommandés}}{\text{Films recommandés}}
$$

---

### 🔁 Rappel

> Proportion de films appréciés par l'utilisateur qui ont été correctement recommandés.

$$
\text{Rappel} = \frac{\text{Films correctement recommandés}}{\text{Films réellement appréciés}}
$$

---

### 🏅 F1-score

> Moyenne harmonique entre la précision et le rappel.  
> Fournit une mesure équilibrée entre ces deux indicateurs.

$$
\text{F1-score} = 2 \times \frac{\text{Précision} \times \text{Rappel}}{\text{Précision} + \text{Rappel}}
$$

---

Ces métriques permettent de comparer les recommandations aux préférences réelles des utilisateurs, et de mesurer l'efficacité du système hybride.

In [105]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Fonction d'évaluation de la performance
def evaluate_performance(user_id, recommended_movies, top_n=4):
    """Évalue la performance du système de recommandation en console."""
    
    # Récupérer les films aimés par l'utilisateur dans les données (rating >= 4)
    test_movies = ratings[ratings['userId'] == user_id]
    test_movies = test_movies[test_movies['rating'] >= 4.0]
    
    # Récupérer les IDs des films recommandés
    recommended_movie_ids = []
    for title, _ in recommended_movies:
        match = movies[movies['title'] == title]
        if not match.empty:
            recommended_movie_ids.append(match.iloc[0]['movieId'])

    # Récupérer les IDs des films que l'utilisateur a réellement aimés
    actual_movie_ids = test_movies['movieId'].tolist()

    # Créer les vecteurs y_true et y_pred
    y_true = [1 if movie_id in actual_movie_ids else 0 for movie_id in recommended_movie_ids]
    y_pred = [1] * len(recommended_movie_ids)  # tous les films recommandés sont des positifs prédits

    # Vérification : éviter division par zéro si aucune correspondance
    if not y_true or all(val == 0 for val in y_true):
        print("⚠️ Aucune correspondance entre recommandations et films réellement aimés.")
        return 0.0, 0.0, 0.0

    # Calcul des métriques
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return precision, recall, f1

# Utilisation : choisir un utilisateur à tester
user_id = 1
recommended_movies = HybridRecommender(user_id)

# Évaluer les performances
precision, recall, f1 = evaluate_performance(user_id, recommended_movies)

# Affichage dans la console
print("\n📊 Évaluation de la performance du système de recommandation")
print(f"Utilisateur testé : {user_id}")
print(f"Nombre de films recommandés : {len(recommended_movies)}")
print(f"🎯 Précision : {precision:.2f}")
print(f"🔁 Rappel    : {recall:.2f}")
print(f"🏅 F1-score  : {f1:.2f}")


📊 Évaluation de la performance du système de recommandation
Utilisateur testé : 1
Nombre de films recommandés : 3
🎯 Précision : 0.33
🔁 Rappel    : 1.00
🏅 F1-score  : 0.50


###  Conclusion

Le système hybride permet de mieux répondre aux préférences des utilisateurs que les méthodes classiques.

- Le filtrage collaboratif permet de capter des relations subtiles entre utilisateurs.
- Le contenu permet de recommander même aux nouveaux utilisateurs.
- La combinaison des deux améliore la robustesse et la précision.

🚀 Perspectives :
- Intégrer la factorisation matricielle (SVD)
- Analyser les critiques textes des films
- Affiner le poids entre les méthodes